In [1]:
# importing libraries
import numpy as np
import cv2
import time
import sys
import matplotlib.pyplot as plt
import os

#### Dataset Paths

In [2]:
dataset_1 = "C:/Users/May/Desktop/SURA24/datasets/1"
dataset_1_jet = "C:/Users/May/Desktop/SURA24/datasets/1/cmapjet"
dataset_1_brg = "C:/Users/May/Desktop/SURA24/datasets/1/cmapbrg"
dataset_1_jet_foc = "C:/Users/May/Desktop/SURA24/datasets/1/cmapjet_focused"
dataset_1_jet_f2 = "C:/Users/May/Desktop/SURA24/datasets/1/cmapjet_f2"
dataset_1_jet_f3 = "C:/Users/May/Desktop/SURA24/datasets/1/cmapjet_f3"

dataset_indoor_1_jet = "C:/Users/May/Desktop/SURA24/datasets/Indoor Images/1/cmapjet"
dataset_indoor_1 = "C:/Users/May/Desktop/SURA24/datasets/Indoor Images/1"
dataset_8 = "C:/Users/May/Desktop/SURA24/datasets/8"
dataset_8_jet = "C:/Users/May/Desktop/SURA24/datasets/8/cmapjet"
dataset_8_brg = "C:/Users/May/Desktop/SURA24/datasets/8/cmapbrg"
dataset_8_gray = "C:/Users/May/Desktop/SURA24/datasets/8/cmapgray"
dataset_3 = "C:/Users/May/Desktop/SURA24/datasets/3"
dataset_3_jet = "C:/Users/May/Desktop/SURA24/datasets/3/cmapjet"
dataset_3_brg = "C:/Users/May/Desktop/SURA24/datasets/3/cmapbrg"
dataset_4 = "C:/Users/May/Desktop/SURA24/datasets/4"
dataset_4_jet = "C:/Users/May/Desktop/SURA24/datasets/4/cmapjet"
dataset_4_brg = "C:/Users/May/Desktop/SURA24/datasets/4/cmapbrg"
dataset_4_gray = "C:/Users/May/Desktop/SURA24/datasets/4/cmapgray"
dataset_5 = "C:/Users/May/Desktop/SURA24/datasets/5"
dataset_5_jet = "C:/Users/May/Desktop/SURA24/datasets/5/cmapjet"
dataset_5_brg = "C:/Users/May/Desktop/SURA24/datasets/5/cmapbrg"
dataset_5_gray = "C:/Users/May/Desktop/SURA24/datasets/5/cmapgray"
dataset_16 = "C:/Users/May/Desktop/SURA24/datasets/16"
dataset_16_jet = "C:/Users/May/Desktop/SURA24/datasets/16/cmapjet"
dataset_16_brg = "C:/Users/May/Desktop/SURA24/datasets/16/cmapbrg"
dataset_16_gray = "C:/Users/May/Desktop/SURA24/datasets/16/cmapgray"
synthetic_1 = "C:/Users/May/Desktop/SURA24/datasets/1/output_crops_jpg"
synthetic_8 = "C:/Users/May/Desktop/SURA24/datasets/8/output_crops_jpg"

#### Helpers 

In [3]:
# adopted from imutils library (https://github.com/jrosebr1/imutils/)
def grab_contours(cnts): 
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

# this function returns a cropped version of the stiched image which doesnot contain black borders
# Note: I have not considered all the edge cases here
def improve_stitching_result(im):
    img = im.copy()
    im[np.where((im==[0,0,0]).all(axis=2))] = [255,255,255]
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 250, 255, 0)

    # Find contours based on OpenCV version
    contours = None
    hierarchy = None
    if cv2.__version__.startswith('3.'):
        im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    elif cv2.__version__.startswith('4.'):
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    else:
        raise NotImplementedError("OpenCV version not supported")

    if contours is not None and len(contours) != 0:
        # find the biggest contour (c) by the area
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)

        return img[0:y+h, 0:x+w, :]

    # Handle case where no valid contours are found
    return img


In [4]:
def crop_black_borders(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Binary mask where white pixels represent non-black areas
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Get the bounding box of the largest contour
    if contours:
        x, y, w, h = cv2.boundingRect(contours[0])
        for cnt in contours:
            x_, y_, w_, h_ = cv2.boundingRect(cnt)
            x = min(x, x_)
            y = min(y, y_)
            w = max(w, w_)
            h = max(h, h_)
        
        # Crop the image to the bounding box
        cropped = image[y:y+h, x:x+w]
        return cropped
    else:
        # If no contours were found, return the original image
        return image

def display_update(img1_orig, img2_orig, result):
    plt.subplot(3, 2, 1)
    plt.imshow(cv2.cvtColor(img1_orig, cv2.COLOR_BGR2RGB))
    plt.title("Stitched Image")

    plt.subplot(3, 2, 2)
    plt.imshow(cv2.cvtColor(img2_orig, cv2.COLOR_BGR2RGB))
    plt.title("Next Image")

    plt.subplot(3, 2, 3)
    plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    plt.title("Stitched Result")

    plt.tight_layout()
    plt.show()

#### Main Pipeline

In [6]:
def stitch(img1, img2, int, dir):

    img1_orig = cv2.imread(img1)
    img2_orig = cv2.imread(img2)
    result = None
    result_path = None

    begin = time.perf_counter()

    img1 = cv2.cvtColor(img1_orig, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2_orig, cv2.COLOR_BGR2GRAY)

    # Creating a SIFT object
    sift = cv2.SIFT_create()

    # Finding keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Creating BFMatcher object
    if des1 is not None and des2 is not None:
        bf = cv2.BFMatcher(cv2.NORM_L2)
        matches = bf.knnMatch(des1, des2, k=2)

        # Apply ratio test
        good_matches = []
        for m_n in matches:
            if len(m_n) == 2:
                m, n = m_n
                if m.distance / n.distance < 0.8:
                    good_matches.append((m.trainIdx, m.queryIdx))
        
        kp1 = np.float32([kp.pt for kp in kp1])
        kp2 = np.float32([kp.pt for kp in kp2])

        reprojThresh = 4.0
        if len(good_matches) > 4:
            pts1 = np.float32([kp1[i] for (_, i) in good_matches])
            pts2 = np.float32([kp2[i] for (i, _) in good_matches])

            H, status = cv2.findHomography(pts1, pts2, cv2.RANSAC, reprojThresh)

            try: 
                result = cv2.warpPerspective(img1_orig, H, (img1_orig.shape[1] + img2_orig.shape[1], max(img1_orig.shape[0], img2_orig.shape[0])))
                result[0:img2_orig.shape[0], 0:img2_orig.shape[1]] = img2_orig
                
                result = improve_stitching_result(result)
                result = crop_black_borders(result)
    
                # display_update(img1_orig, img2_orig, result)
                
            except:
                print("Not enough good matches were found this iteration; skipped")

        else: 
            print("Not enough good matches were found.")
            
    else: 
        print("No keypoints detected.")
        good_matches = None

    if (result is not None): 
        try:
           os.makedirs(dir)
        except FileExistsError:
           pass
        result_path = f"{dir}/{int}.JPG"
        cv2.imwrite(result_path, result)
    return (good_matches, result_path)
    


In [7]:
# Creates a simulated "ground truth" image by applying the RGB 
# image transformations to their corresponding thermal images
def ground_truth(img1, img2, ther1, ther2, int, dir):

    img1_orig = cv2.imread(img1)
    img2_orig = cv2.imread(img2)
    ther1_orig = cv2.imread(ther1)
    ther2_orig = cv2.imread(ther2)
    result = None
    result_path = None

    begin = time.perf_counter()

    img1 = cv2.cvtColor(img1_orig, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2_orig, cv2.COLOR_BGR2GRAY)
    ther1 = cv2.cvtColor(ther1_orig, cv2.COLOR_BGR2GRAY)
    ther2 = cv2.cvtColor(ther2_orig, cv2.COLOR_BGR2GRAY)

    # Creating a SIFT object
    sift = cv2.SIFT_create()

    # Finding keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Creating BFMatcher object
    if des1 is not None and des2 is not None:
        bf = cv2.BFMatcher(cv2.NORM_L2)
        matches = bf.knnMatch(des1, des2, k=2)

        # Apply ratio test
        good_matches = []
        for m_n in matches:
            if len(m_n) == 2:
                m, n = m_n
                if m.distance / n.distance < 0.8:
                    good_matches.append((m.trainIdx, m.queryIdx))
        
        kp1 = np.float32([kp.pt for kp in kp1])
        kp2 = np.float32([kp.pt for kp in kp2])

        reprojThresh = 4.0
        if len(good_matches) > 4:
            pts1 = np.float32([kp1[i] for (_, i) in good_matches])
            pts2 = np.float32([kp2[i] for (i, _) in good_matches])

            H, status = cv2.findHomography(pts1, pts2, cv2.RANSAC, reprojThresh)

            try: 
                result = cv2.warpPerspective(ther1_orig, H, (ther1_orig.shape[1] + ther2_orig.shape[1], max(ther1_orig.shape[0], ther2_orig.shape[0])))
                result[0:ther2_orig.shape[0], 0:ther2_orig.shape[1]] = ther2_orig
                
                result = improve_stitching_result(result)
                result = crop_black_borders(result)
    
                # display_update(ther1_orig, ther2_orig, result)
                
            except:
                print("Not enough good matches were found this iteration; skipped")

        else: 
            print("Not enough good matches were found.")
            
    else: 
        print("No keypoints detected.")
        good_matches = None

    if (result is not None): 
        try:
           os.makedirs(dir)
        except FileExistsError:
           pass
        result_path = f"{dir}/{int}.JPG"
        cv2.imwrite(result_path, result)
    return (good_matches, result_path)
    


In [8]:
# example useage 

# img1 = "/datasets/1/20240611_070653_687_R.JPG"
# img2 = "/datasets/1/20240611_070656_687_R.JPG"
# ther1 = "/datasets/1/cmapjet/jet_0.JPG"
# ther2 = "/datasets/1/cmapjet/jet_1.JPG"
# stitch(ther2, ther1, 0)
# ground_truth(img2, img1, ther2, ther1, 0)

MIT License

Copyright (c) [year] [fullname]

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.